In [1]:
%matplotlib inline
# ---- settings ----
import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../samples')
import paf_loader
from os.path import join, isdir
import numpy as np
import cv2
from cselect import color as cs

from mvpose.data import umpm

root = join(Settings['data_root'], 'pak')
user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']

# **************************** #
FRAME = 0
# **************************** #

X, Y, Calib = umpm.get(root, 'p2_free_1', user, pwd)

Im = np.array([X[0][FRAME], X[1][FRAME], X[2][FRAME], X[3][FRAME]])

heatmaps, pafs = paf_loader.load_confidence_map_and_paf('umpm', Im, FRAME)


/home/julian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


UMPM


In [2]:
from mvpose.data.default_limbs import DEFAULT_LIMB_SEQ, \
    DEFAULT_MAP_IDX, DEFAULT_SENSIBLE_LIMB_LENGTH
from mvpose.algorithm.peaks2d import Candidates2D
from mvpose.algorithm.triangulation import Triangulation
from mvpose.algorithm.limbs3d import Limbs3d
from time import time

max_epi_distance = 10

limbSeq = DEFAULT_LIMB_SEQ
sensible_limb_length = DEFAULT_SENSIBLE_LIMB_LENGTH
limbMapIdx = DEFAULT_MAP_IDX

# -------- step 1 --------
# calculate 2d candidates
# ------------------------
_start = time()
cand2d = Candidates2D(heatmaps, Calib)
_end = time()
print('step 1: elapsed', _end - _start)

# -------- step 2 --------
# triangulate 2d candidates
# ------------------------
_start = time()
triangulation = Triangulation(cand2d, max_epi_distance)
_end = time()
print('step 2: elapsed', _end - _start)

step 1: elapsed 1.1789746284484863
step 2: elapsed 0.014447689056396484


In [55]:

@jit([float64(float64[:,:,])], nopython=True)
def do_stuff(L):
    Mask = (L >= 1) * 1
    
    x,y = np.nonzero(Mask)
    print(x)
    print(y)
    
    #XY = np.transpose(np.nonzero(Mask).astype('int64'))
    
    return 0.0


L = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [1, 1, 0]
], 'float64')

do_stuff(L)

[0 1 2 2]
[2 1 0 1]


0.0

In [89]:
from mvpose.plot.limbs import draw_vector_field_readonly
import numba as nb
from numba import jit, float64, boolean
from mvpose.compiler.numba_types import float64_2d_const
import numpy.linalg as la


#@jit([float64[:,:](float64[:,:,], float64[:,:,], float64_2d_const, float64_2d_const, boolean)], nopython=True)
@jit([float64[:,:](float64[:,:,], float64[:,:,], float64[:,:,], float64[:,:,])], nopython=True)
def calculate_line_integral_elementwise(candA, candB, mapx, mapy):
    """
        calculates the line integral for points element-wise
    :param candA: [ (x,y), (x,y), ...]
    :param candB: [ (x,y), (x,y), ...]
    :param mapx: w x h
    :param mapy: w x h
    :param normalize:
    :return:
    """
    assert mapx.shape == mapy.shape
    assert len(candA.shape) == 2
    assert len(candB.shape) == 2
    assert candA.shape[1] == 2
    assert candB.shape[1] == 2
    assert len(mapx.shape) == 2
    mid_num = 10
    nA = len(candA)
    nB = len(candB)
    assert nA == nB

    h, w = mapx.shape
    score_mid = np.zeros((h,w,2))
    score_mid[:,:,0] = mapx
    score_mid[:,:,1] = mapy

    W = np.zeros((nA, 1))

    for i in range(nA):
        d = np.subtract(candB[i], candA[i])
        norm = la.norm(d)
        if norm == 0:
            continue
        d = np.divide(d, norm)

        iterX = np.linspace(candA[i][0], candB[i][0], mid_num)
        iterY = np.linspace(candA[i][1], candB[i][1], mid_num)

        for x, y in zip(iterX, iterY):
            x_ = min(int(round(x)), w-1)
            y_ = min(int(round(y)), h-1)
            Lc = score_mid[y_, x_]
            W[i] += Lc @ d

    return W/mid_num

In [92]:
import numpy.linalg as la
from scipy.special import comb
import mvpose.geometry.geometry as gm
from mvpose.plot.limbs import draw_vector_field
#from mvpose.algorithm.limbs3d import calculate_line_integral_elementwise
#from mvpose.algorithm.limbs3d import clamp_to_1




peaks3d = triangulation.peaks3d_weighted
Pafs = pafs

# ===================================================
_start = time()
# ===================================================
n_cameras, h, w, _ = Pafs.shape
n_limbs = len(limbSeq)
assert n_cameras == len(Calib)
assert len(limbSeq) == len(sensible_limb_length)
CAMERA_NORM = comb(n_cameras, 2)  # this is needed to make the 3d pafs be between -1 .. 1
limbs3d = [None] * n_limbs

for idx, ((a, b), (length_min, length_max), (pafA, pafB)) in \
        enumerate(zip(limbSeq, sensible_limb_length, limbMapIdx)):
    # 3d peaks are setup as follows:
    #       (x,y,z,score1,score2)
    candA3d = peaks3d[a]
    candB3d = peaks3d[b]

    nA = len(candA3d)
    nB = len(candB3d)

    W = np.zeros((nA, nB))

    if nA > 0 and nB > 0:
        for cid, cam in enumerate(Calib):
            
            
            K, rvec, tvec, distCoef = gm.get_camera_parameters(cam)

            U = Pafs[cid, :, :, pafA]; U.setflags(write=False)
            V = Pafs[cid, :, :, pafB]; V.setflags(write=False)

            ptsA2d, maskA = gm.reproject_points_to_2d(
                candA3d[:, 0:3], rvec, tvec, K, w, h, distCoef=distCoef, binary_mask=True)
            ptsB2d, maskB = gm.reproject_points_to_2d(
                candB3d[:, 0:3], rvec, tvec, K, w, h, distCoef=distCoef, binary_mask=True)
            maskA = maskA == 1
            maskB = maskB == 1
            
            ptA_candidates = []
            ptB_candidates = []
            pair_candidates = []
            
            for i, (ptA, ptA3d, is_A_on_screen) in enumerate(zip(ptsA2d, candA3d, maskA)):
                if not is_A_on_screen:
                    continue
                for j, (ptB, ptB3d, is_B_on_screen) in enumerate(zip(ptsB2d, candB3d, maskB)):
                    if not is_B_on_screen:
                        continue
                    distance = la.norm(ptA3d[0:3] - ptB3d[0:3])
                    if length_min < distance < length_max:
                        ptA_candidates.append(ptA)
                        ptB_candidates.append(ptB)
                        pair_candidates.append((i, j))

            if len(ptA_candidates) > 0:
                line_int = calculate_line_integral_elementwise(
                    np.array(ptA_candidates), np.array(ptB_candidates), U, V)
                line_int = np.clip(line_int, a_min=-1, a_max=1)
                assert len(line_int) == len(pair_candidates)
                line_int = np.squeeze(line_int / CAMERA_NORM)
                if len(line_int.shape) == 0:  # this happens when line_int.shape = (1, 1)
                    line_int = np.expand_dims(line_int, axis=0)
                for score, (i, j) in zip(line_int, pair_candidates):
                    W[i, j] += score
            


    limbs3d[idx] = W

# ===================================================
_end = time()
print('elapsed', _end - _start)
# ===================================================

elapsed 0.6229281425476074
